# [모듈 1.2] 이미지 분류를 위한 폴더 구조 생성
Download | **Structure** | Preprocessing | Train Model (4단계 중의 2/4)

# 이미지 분류를 위한 폴더 구조
___

- TensorFlow 및 PyTorch, MXNet 대부분의 프레임워크는 수정을 통해 모든 파일 구조의 데이터를 수용할 수 있습니다.
- 하지만, 이 같은 프레임워크가 모두 수용 가능한 파일 구조를 가지게 되면 모델 개발이 빨라 지게 됩니다. 
- 기본적으로 대부분의 프레임워크는 아래에 설명된 파일 구조에서 이미지 데이터를 찾습니다.
이 노트북에서는 아래와 같은 구조를 만들어 보겠습니다.

```
+-- train
|   +-- class_A
|       +-- filename.jpg
|       +-- filename.jpg
|       +-- filename.jpg
|   +-- class_B
|       +-- filename.jpg
|       +-- filename.jpg
|       +-- filename.jpg
|
+-- val
|   +-- class_A
|       +-- filename.jpg
|       +-- filename.jpg
|       +-- filename.jpg
|   +-- class_B
|       +-- filename.jpg
|       +-- filename.jpg
|       +-- filename.jpg
|
+-- test
|   +-- class_A
|       +-- filename.jpg
|       +-- filename.jpg
|       +-- filename.jpg
|   +-- class_B
|       +-- filename.jpg
|       +-- filename.jpg
|       +-- filename.jpg
```
- COCO 데이터셋은 위와 같이 구성되지 않으므로 주석 데이타를 사용하여 위의 패턴과 일치하도록 만들겁니다.
- **새 디렉토리 구조가 생성되면 원하는 프레임워크의 데이터 로드 툴을 사용하여 이미지 데이터에 대한 로딩 및 변환을 할 수 있습니다.** 
- 보통의 많은 예제들은 이러한 과정이 생략이 되어 있습니다. 하지만 현업에서 실제 작업을 할 시에는 이러한 과정을 해야하기에 여기서 배워 봄니다.

# 노트북 요약
---


- 이 노트북에서는 샘플의 주석 및 카테고리 데이타를 로딩 합니다.
- 데이터를 훈련, 검증 및 테스트 세트로 분할합니다. 
- 그런 다음 Python을 사용하여 새 폴더 구조를 만들고 파일을 올바른 세트 및 레이블 폴더에 복사합니다.

# 0. 환경 설정

In [1]:
import shutil
import pickle
import numpy as np
from tqdm import tqdm
from pathlib import Path

In [2]:
%load_ext autoreload
%autoreload 2

# 1. 샘플 주석 (2750개) 및 카테고리 레이블 로딩
___
`sample_annos` 및 `category_labels` 파일은 이 시리즈 `1.1_download_data.ipynb`의 첫 번째 노트북에서 생성되었습니다. 여기에서 코드를 실행하기 전에 해당 노트북을 실행해야 합니다.

In [3]:
with open("pickled_data/sample_annos.pickle", "rb") as f:
    sample_annos = pickle.load(f)

with open("pickled_data/category_labels.pickle", "rb") as f:
    category_labels = pickle.load(f)

# 2. 데이터 셋을 훈련, 검증, 테스트 로 분리
___
- 데이터를 학습, 검증 및 테스트 분할로 나누어야 합니다. 
- 일반적인 분할 비율은 80/10/10입니다. 
- 우리의 이미지 분류 알고리즘은 처음 80%(훈련)에 대해 훈련하고 다음 10%(검증)로 각 에폭(Epoch)의 성능을 평가 합니다.
- 마지막 10%(테스트)를 사용하여 모델의 최종 정확도 결과를 제공합니다. 
- 데이터를 분할하기 전에 분할 간의 클래스 분포가 대략 비례하도록 무작위로 섞는 것이 중요합니다.

In [4]:
np.random.seed(0)
image_ids = sorted(list(sample_annos.keys())) # 이미지 ID 리스트 추출
np.random.shuffle(image_ids) 

first_80 = int(len(image_ids) * 0.8)
next_10 = int(len(image_ids) * 0.9)
train_ids, val_ids, test_ids = np.split(image_ids, [first_80, next_10])

print("train_shape: \n", train_ids.shape)
print("val_ids_shape: \n", val_ids.shape)
print("test_ids_shape: \n", test_ids.shape)

print("val_ids: \n", val_ids[0:5])
print("train_ids: \n", train_ids[0:5])
print("test_ids: \n", test_ids[0:5])


train_shape: 
 (2200,)
val_ids_shape: 
 (275,)
test_ids_shape: 
 (275,)
val_ids: 
 [581722 181697 115859 446764 165141]
train_ids: 
 [168879 382564 210275 571938 518455]
test_ids: 
 [ 85288 466210 581855 324937 581749]


# 3. 새 폴더 구조를 만들고 이미지 파일을 복사합니다.
___
아래와 같은 순서로 코드가 동작 합니다.
- data_structured 생성
    - train, val, test 폴더 생성
        - 카테고리 폴더 생성 
            - 예: category_dir:  data_structured/train/giraffe
        - 소스 파일을 타겟 경로로 복사                
            - 소스파일 지정    
                - 예: source_path:  data_sample_2750/000000132015.jpg
            - 타겟경로 지정    
                - 예: target_path:  data_structured/train/giraffe/000000132015.jpg
            - 복사


In [5]:
import os
import shutil

unstruct_dir = Path("data_sample_2750") 
struct_dir = Path("data_structured") # 구조화된 폴더 이름

if os.path.isdir(struct_dir): # 폴더 존재하면 삭제
    shutil.rmtree(struct_dir)
    print(f"{struct_dir} is deleted")

struct_dir.mkdir(exist_ok=True, parents=True) # 폴더 생성    
print(f"{struct_dir} is created")    


data_structured is created


In [6]:
# train, val, test 세번 반복
for name, split in zip(["train", "val", "test"], [train_ids, val_ids, test_ids]):
    split_dir = struct_dir / name
    split_dir.mkdir(exist_ok=True) # train, val, test 폴더 생성
    for image_id in tqdm(split):
        category_dir = split_dir / f'{category_labels[sample_annos[image_id]["category_id"]]}'
        # print("category_dir: ", category_dir)
        category_dir.mkdir(exist_ok=True) # category 폴더 생성
        source_path = (unstruct_dir / sample_annos[image_id]["file_name"]).as_posix()
        target_path = (category_dir / sample_annos[image_id]["file_name"]).as_posix()
#         print("source_path: ", source_path)
#         print("target_path: ", target_path)        
        shutil.copy(source_path, target_path)
#         break

100%|██████████| 275/275 [00:00<00:00, 5442.29it/s]


###  만들어진 폴더를 확인

In [7]:
from pathlib import Path
 
def listdirs(rootdir):
    for path in Path(rootdir).iterdir():
        if path.is_dir():
            print(path)
            listdirs(path)
 
rootdir = 'data_structured'
listdirs(rootdir)

data_structured/test
data_structured/test/giraffe
data_structured/test/zebra
data_structured/test/frog
data_structured/test/elephant
data_structured/test/bear
data_structured/test/sheep
data_structured/test/cat
data_structured/test/cow
data_structured/test/bird
data_structured/test/dog
data_structured/test/horse
data_structured/train
data_structured/train/giraffe
data_structured/train/zebra
data_structured/train/frog
data_structured/train/elephant
data_structured/train/bear
data_structured/train/sheep
data_structured/train/cat
data_structured/train/cow
data_structured/train/bird
data_structured/train/dog
data_structured/train/horse
data_structured/val
data_structured/val/giraffe
data_structured/val/zebra
data_structured/val/frog
data_structured/val/elephant
data_structured/val/bear
data_structured/val/sheep
data_structured/val/cat
data_structured/val/cow
data_structured/val/bird
data_structured/val/dog
data_structured/val/horse


# 4. 다음 과정
- 선택한 프레임워크에서 이미지를 쉽게 로드할 수 있으므로 다음 단계는 프레임워크를 선택하는 것입니다. 
    - (1) SageMaker의 내장 알고리즘 
    - (2) TensorFlow
    - (3) PyTorch를 


- 자세히 알고 싶은 프레임워크에 따라 다음 노트북을 선택할 수 있습니다. 데이터가 프레임워크에 로드되면 전처리, 파일 형식, 변환 및 데이터 증강을 다룹니다.